In [13]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from keras.applications.inception_v3 import InceptionV3
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D
from keras import backend as K
from keras.layers import Input
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd
from skimage.transform import resize
from sklearn.preprocessing import OneHotEncoder
import threading

In [2]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [17]:
df = pd.read_csv('./train.csv')
df_test = pd.read_csv('./test.csv')
y = df.values[:, 0]
# y = df.values[:, 0].reshape(-1, 1)
# y = OneHotEncoder().fit_transform(y).toarray()
X_ori = np.delete(df.values, 0, 1).reshape((-1, 28, 28))
X_test_ori = np.delete(df_test.values, 0, 1).reshape((-1, 28, 28))

In [8]:
cpu_num = os.cpu_count()
X_ori_list = np.array_split(np.arange(X_ori.shape[0]), cpu_num)


tmp = X_ori.repeat(3, axis=2).reshape(-1, 28, 28, 3)
X = tmp.repeat(3, axis=1).repeat(3, axis=2).reshape(-1, 84, 84, 3)

tmp = X_test_ori.repeat(3, axis=2).reshape(-1, 28, 28, 3)
X_test = tmp.repeat(3, axis=1).repeat(3, axis=2).reshape(-1, 84, 84, 3)

print('X shape: {}'.format(X.shape))
print('X_test shape: {}'.format(X_test.shape))

X shape: (60000, 84, 84, 3)
X_test shape: (10000, 84, 84, 3)


In [14]:
train_datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    channel_shift_range=10,
    horizontal_flip=True,
    fill_mode='nearest')
train_batches = train_datagen.flow(X, y, batch_size=128)

In [22]:
base_model = InceptionV3(
    input_tensor=None,
    input_shape=(84, 84, 3),
    weights='imagenet',
    include_top=False)

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

for layer in base_model.layers:
    layer.trainable = False

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'])

model.fit(X, y, batch_size=128, epochs=10, verbose=1)

history_tl = model.fit_generator(
    generator=train_batches,
    steps_per_epoch=800,
    epochs=2,
    verbose=1)

model.evaluate(X, y)

Epoch 1/10
60000/60000 [==============================] - 24s 395us/step - loss: 1.2481 - acc: 0.5855
Epoch 2/10
60000/60000 [==============================] - 18s 295us/step - loss: 1.0469 - acc: 0.6411
Epoch 3/10
60000/60000 [==============================] - 17s 291us/step - loss: 0.9724 - acc: 0.6614
Epoch 4/10
60000/60000 [==============================] - 17s 287us/step - loss: 0.9254 - acc: 0.6771
Epoch 5/10
60000/60000 [==============================] - 17s 287us/step - loss: 0.8834 - acc: 0.6901
Epoch 6/10
60000/60000 [==============================] - 17s 286us/step - loss: 0.8595 - acc: 0.6989
Epoch 7/10
60000/60000 [==============================] - 17s 285us/step - loss: 0.8188 - acc: 0.7118
Epoch 8/10
60000/60000 [==============================] - 17s 286us/step - loss: 0.8017 - acc: 0.7183
Epoch 9/10
60000/60000 [==============================] - 17s 287us/step - loss: 0.7759 - acc: 0.7257
Epoch 10/10
60000/60000 [==============================] - 17s 286us/step - loss: 

ValueError: Error when checking target: expected dense_12 to have shape (1,) but got array with shape (10,)

In [21]:
y_test = model.predict(X_test)
answer = pd.DataFrame()
answer['id'] = range(len(y_test))
answer['label'] = np.argmax(y_test, axis=1).astype(int)
answer.to_csv('answer.csv', index=False)